# Tutorial

This module will guide you through some minimal steps in looking at and manipulating existing modules in the package. 



In [8]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import os

def run():
    path = Path.cwd()
    while len(path.name) and path.name != 'codes':
        path = path.parent

    if len(path.name):
        os.chdir(path)
    else:
        raise ValueError('Cannot find the root directory of the project.')

run()
from path_settings import *
import joblib
from pathlib import Path
from importlib import reload
from agents import Agent
from tasks import akam_tasks as ts
from utils import *  # get_current_file_name, goto_root_dir
import pprint
import CustomMapper.CustomMapper as c
import torch
import pickle
import pandas as pd
import numpy as np
import config as co
from collections import defaultdict
import matplotlib.pyplot as plt
import torch.nn as nn
import time
from agents.DynamicSystems import ISNNet
from agents.ComplexAgent import DynamicAgent
import copy
import plotly.express as px

pp = pprint.PrettyPrinter(indent=4, depth = 4)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Step 1: simulate some datasets

In my setup, I organize datasets in the CustomMapper data type. The datatype basically links a cognitive model with an RT model, setting up parameters and stuff. 

In [ ]:
# Define agent and task. 
# This is based on Ji-an's codes. The end product is consist of an array of stims, choices, and rewards
# it will automatically save the data in the folder specified in the config file. 

tasks = {
    # "PRL": ts.Two_step(com_prob=1),  # common transition always happens
    # "RTS": ts.Two_step(), # reversal two-stage, probabilistic transition 
    # "NTS": ts.Two_step(rew_gen="trans_rev"), 
    "walk": ts.Two_step(rew_gen="walks"),
}

N_blocks = 1000
N_trials = 100
device = "cpu"
seed = 0
agents = {}
config = {}
cog_types = ["MB0s", "LS0", "MB0", "MB1"]


for cog_type in cog_types:
    cog_config = co.sim_config_from_inputs(False, 
                                        agent_type="RTSCog", # calls a cognitive agent for the reduced two step task. 
                                        cog_type=cog_type, # agent type 
                                        device=device, 
                                        seed=seed,
                                        num_blocks=N_blocks,
                                        num_trials=N_trials,
                                        exp_folder=0)
    config[f'walks_{cog_type}'] = cog_config
    agents[f'walks_{cog_type}'] = Agent(cog_config['agent_type'], config = cog_config)

for cog_type in cog_types: 
    agents[f'walks_{cog_type}'].simulate(tasks["walk"], config[f'walks_{cog_type}'], save = True)


Simulating cog agent MB0s with params [0.5, 5.0]
n_blocks 1000 n_trials 100 sim_seed 0 sim_exp_name simulated_Akam_RTS additional_name 
Simulating cog agent LS0 with params [0.1, 5.0]
n_blocks 1000 n_trials 100 sim_seed 0 sim_exp_name simulated_Akam_RTS additional_name 
Simulating cog agent MB0 with params [0.5, 5.0]
n_blocks 1000 n_trials 100 sim_seed 0 sim_exp_name simulated_Akam_RTS additional_name 
Simulating cog agent MB1 with params [0.5, 0.5, 5.0]
n_blocks 1000 n_trials 100 sim_seed 0 sim_exp_name simulated_Akam_RTS additional_name 


Now, onto defining the CustomMapper. 

It was designed to hold everything, from the cognitive datasets to the RT models, and to your RNN model. However I only used it in the first iterations in the project (later I just defined new trainers and datasets). You are welcome to modify the codes a bit to clean everything up and make everything uniform and tidy. 


In [ ]:
RTRNN_config = {
    "task": "PRL_Bartolo", 
    "dt": 0.02, # 0.02s time step, as in Jaffe et al. This is also used in the DDM simulation of RTs. 
    "T": 200, # 1s upper for RT for the DDM 
    "cog_data": SIM_SAVE_PATH / 'simulated_Akam_RTS' / 'LS0_seed0.pkl',
    "bias": 0.01, 
    "ndt_s": 0.01, 
    "ndt_mu": 0.1, 
    "driftscale": 2, 
    "redo_choices": False,
    "trainer_type": "RTRNNTrainer",
    "model_specs": {
        "model_name": "RTRNN", 
        "model_path": "Network_models.RTRNN",
        "model_params":{ # model setup params, NOTE that recurrence_per_trial is how many RNN steps per trial
            "batch_size": 20, "input_size": 6, "output_size": 2, "hidden_size": 16, "recurrence_per_trial": 2,
            "cell_type":"GRU", "trial_output_hidden": 0, "trial_output_hidden_size": [], # if want hidden output layers (deep output, then give a list)
            "trial_output_hidden_nonlinearities": [], "pad_zeros":0, "last_step": False
        },
    },
    "rt_weight": 0.01, # weight for the RT loss. 
    "device": "cuda", 
    # "scheduler": { # highly optional
    #     "name": "ReduceLROnPlateau",
    #     "params": {
    #         "mode": "min",
    #         "factor": 0.1,
    #         "patience": 100,
    #         "min_lr": 5e-4,
    #         "cooldown": 50, 
    #     },
    # }, 
    "optimizer_specs":{
        "optimizer_name": "Adam",
        "optimizer_params": {
            "lr": 0.0005,
        },
    },
    "training_config":{
        "task_id": "1.1", # as opposed to "Task-DyVA". You may additionally put in later datasets here to convert to RT datasets.
        "task_config": {
            "task": "PRL_Bartolo",
        }
    },  
    "save_path": MODEL_SAVE_PATH ,
    "check_path": MODEL_SAVE_PATH,
    "log_path": LOG_PATH, # These are to be modified when the big configs gets defined
}

for agent in ["MB0s"]:#, "LS0", "MB0", "MB1"]:
    RTRNN_config["cog_data"] = SIM_SAVE_PATH / 'simulated_Akam_RTS' / f'{agent}_seed0.pkl'
    experiment = c.CustomMapper(RTRNN_config) # the RT is added right away. 
    experiment.rt_model = "RELOAD"
    pickle.dump(experiment, open(DATA_PATH/f"experiment_{agent}_1.pkl", "wb"))



Block 0
Block 1
Block 2
Block 3
Block 4
Block 5
Block 6
Block 7
Block 8


KeyboardInterrupt: 

In [ ]:
# load the thing like this: 
agent = "MB0s" # This is from cognitive models. 
try: 
    experiment = pickle.load(open(DATA_PATH/f"experiment_{agent}_1.pkl", "rb"))
    experiment.initialise_dataloaders()
    experiment.configs["model_specs"]["model_params"]["batch_size"] = 64
    experiment.initialise_trainer(experiment.configs)
except FileNotFoundError:
    print(f"Experiment for {agent} not found. Please run the simulation first.")

# Step 2: Train the most general form of RTRNNs
see below for example codes to block train some base models for each agent type - for each model try: 
GRU hidden size 8, 16
recurrence_per_trial: 1, 2, 3
zero padding, 0, 1

In [ ]:

# for agent in ["MB0s", "LS0", "MB0", "MB1"]:
#     for hidden_size in [8, 16]:
#         for zero_padding in [0, 1]:
#             for recurrence_per_trial in [1, 2]: 
#                 experiment = pickle.load(open(DATA_PATH/f"experiment_{agent}_1.pkl", "rb"))
#                 experiment.configs["model_specs"]["model_params"]["hidden_size"] = hidden_size
#                 experiment.configs["model_specs"]["model_params"]["pad_zeros"] = zero_padding
#                 experiment.configs["model_specs"]["model_params"]["recurrence_per_trial"] = recurrence_per_trial
#                 experiment.initialise_dataloaders()
#                 experiment.initialise_trainer(experiment.configs)
#                 model_name = f"{agent}_GRU_HS={hidden_size}_N={recurrence_per_trial}_ZP{zero_padding}"
#                 for path in [experiment.configs["save_path"], experiment.configs["check_path"], experiment.configs["log_path"]]:
#                     path = path / model_name 
#                     if not os.path.exists(path):
#                         os.makedirs(path)
#                 experiment.trainer.train(
#                     experiment.dataloaders["train"],
#                     experiment.dataloaders["val"],
#                     epochs = 2000,
#                     save_interval = 500,
#                     reset_interval = 10000,
#                     save_path = experiment.configs["save_path"] / model_name / "model.pth",
#                     check_path = experiment.configs["check_path"] / model_name,
#                     log_path = experiment.configs["log_path"] / model_name,
#                     early_stop = 1000,
#                 )

if you tried training as above, you can load your stuff to visualize as follows to compare model performance

In [ ]:
agent = "LS0"
experiment = pickle.load(open(DATA_PATH/f"experiment_{agent}_0.pkl", "rb"))
hidden = 16
recurrence = 1
zero_padding = 1
experiment.configs["model_specs"]["model_params"]["hidden_size"] = hidden
experiment.configs["model_specs"]["model_params"]["pad_zeros"] = zero_padding
experiment.configs["model_specs"]["model_params"]["recurrence_per_trial"] = recurrence
experiment.initialise_trainer(experiment.configs)
experiment.trainer.load_model(experiment.configs["save_path"]/
                                f"{agent}_RNN_HS={hidden_size}_N={recurrence}_ZP{zero_padding}_experimental_5" / "model.pth")
# plot boxplots, though each key from the dictionary has a different length
rt_performance = pd.DataFrame(rt_performance)
choice_performance = pd.DataFrame(choice_performance)
# plot boxplots with sns
import seaborn as sns
fig, ax = plt.subplots(2, 1, figsize = (10, 10))
sns.boxplot(data = np.log(rt_performance), ax = ax[0])
ax[0].set_xticks(ticks = np.arange(len(rt_performance.columns)),
                 labels = rt_performance.columns, rotation=45)
ax[0].set_title("RT performance of models")
sns.boxplot(data = np.log(choice_performance), ax = ax[1])
ax[1].set_xticks(ticks = np.arange(len(rt_performance.columns)),
                 labels = rt_performance.columns, rotation=45)
ax[1].set_title("Choice performance of models")
plt.tight_layout()


# Step 3: Other RNN models: RTified and Discretised

As mentioned earlier above, I did not persist in using the CustomMapper for everything. Hence in the following examples I show the alternative workflow. 

## 3.1. RTified

The full notebook here is [[RNN_RT.ipynb]]

In [12]:
from Network_models.RTRNN import RTSlowFastRNN
from Network_models.Trainer import SlowFastTrainer

In [ ]:
# RTRNN training on the data!
batch_size = 64
steps = 4
identity_proj = False
configs = {
    "save_path": MODEL_SAVE_PATH / "slow_fast",
    "check_path": MODEL_SAVE_PATH / "slow_fast",
    "log_path": LOG_PATH / "slow_fast",
    "optimizer_specs":{
        "optimizer_name": "Adam",
        "optimizer_params": {
            "lr": 0.001,
        },
    },
    "model_specs":{
        "model_name": "RTSlowFastRNN", 
        "model_path": "Network_models.RTRNN",
        "model_params":{
            "input_size": 6, "n_classes": 2, "h_slow_dim": 16, "h_fast_dim": 16, "fast_steps": steps, "identity_proj": identity_proj,
            "trial_output_hidden": 1, "trial_output_hidden_size": [16,], # if want hidden output layers (deep output, then give a list)
            "trial_output_hidden_nonlinearities": ["ReLU"], 
        },
    },
    "SFloss_specs": {
        "rt_weight": 0.5,  # weight for the RT loss
    }, 

    "device": "cuda", 
    # "scheduler": {
    #     "name": "ReduceLROnPlateau",
    #     "params": {
    #         "mode": "min",
    #         "factor": 0.1,
    #         "patience": 100,
    #         "min_lr": 5e-4,
    #         "cooldown": 50, 
    #     },
    # }, 
    "training_config": {
        "batch_size": 256, 
        "val_ratio": 0.2, 
        "n_trials": 10, 
    }, 
}

# make the dirs
for path in [configs["save_path"], configs["check_path"], configs["log_path"]]:
    Path(path).mkdir(parents=True, exist_ok=True)
rtrnn_trainer = SlowFastTrainer(configs)


In [ ]:
dataloaders = {
    "train": torch.utils.data.DataLoader(
        experiment.dataloaders["train"].dataset,batch_size = batch_size, shuffle=True,)
        
    , "val": torch.utils.data.DataLoader(
        experiment.dataloaders["val"].dataset, batch_size = batch_size, shuffle=False,
        )
    }


NameError: name 'experiment' is not defined

for these models, I would train in a particular way. 

stage 0: no RT loss. 
stage 1: RT loss (train with frozen backbone to warm up the RT module, then unfreeze to fine-tune the cognitive model based on RT information)

In [ ]:
rtrnn_trainer.train_loop(dataloaders["train"], dataloaders["val"], stage = 0, epochs = 20)

In [ ]:
rtrnn_trainer.model.freeze_slow()
rtrnn_trainer.train_loop(dataloaders["train"], dataloaders["val"], stage = 1, epochs = 200)

In [ ]:
rtrnn_trainer.model.unfreeze_slow()
rtrnn_trainer.train_loop(dataloaders["train"], dataloaders["val"], stage = 1, epochs = 200)

## 3.2 Discretised RT

The full notebook here is [[RNN_RT_discretised.ipynb]]

In [ ]:
from Network_models.Trainer import DiscretisedTrainer

# RTRNN training on the data!
batch_size = 64
reward_presentation_time = 0.5
fixation_time = 0.5

# knobs on the computation specs
rsdf = 0 # repeat stim during fixation (default is to present during reward)
rsda = 0 # repeat stim during action
one_hot_input = True
train_h0 = False # set to random initial condition to train the grounding fixed point structure during fixation

dt = 0.1
action_time = 1.5
sigma_action = 0.1 # to smooth the action trains - potentially ease training  
pad_ones = 1

configs = {
    "save_path": MODEL_SAVE_PATH / f"discretised{'_pad_ones' if pad_ones else ''}",
    "check_path": MODEL_SAVE_PATH / f"discretised{'_pad_ones' if pad_ones else ''}",
    "log_path": LOG_PATH / f"discretised{'_pad_ones' if pad_ones else ''}",
    "loss_type": "ce", 
    "optimizer_specs":{
        "optimizer_name": "Adam",
        "optimizer_params": {
            "lr": 0.001,
        },
    },
    "model_specs":{
        "model_name": "RTDiscretisedRNN", 
        "model_path": "Network_models.RTRNN",
        "model_params":{
            "reward_presentation_time": reward_presentation_time, "repeat_stimulus_during_fixation": rsdf,
            "repeat_stimulus_during_action": rsda,
            "fixation_time": fixation_time, "action_time": action_time, 
            "one_hot_input": one_hot_input, "output_size": 3, "dt": dt, 
            "hidden_size": 256, "trial_output_hidden": 1, 
            "hidden_dimensions": [64, ], 
            "hidden_nonlinearities": ["ReLU", ], 
            "train_h0": train_h0, 
            "sigma_action": sigma_action, "batch_size": batch_size, "pad_ones": pad_ones,
        },
    },
    "SFloss_specs": {
        "rt_weight": 0.5,  # weight for the RT loss
    }, 

    "device": "cuda", 
    # "scheduler": False, 
    "scheduler": {
        "name": "ReduceLROnPlateau",
        "params": {
            "mode": "min",
            "factor": 0.1,
            "patience": 100,
            "min_lr": 5e-4,
            "cooldown": 50, 
        },
    }, 
    "training_config": {
        "batch_size": batch_size, 
        "val_ratio": 0.2, 
        "n_trials": 10, 
    }, 
}

# make the dirs
for path in [configs["save_path"], configs["check_path"], configs["log_path"]]:
    Path(path).mkdir(parents=True, exist_ok=True)
rtrnn_trainer = DiscretisedTrainer(configs)


In [ ]:
# The trainer is implemented with a convert_dataset function to go from a cognitive dataset to fully discretised.
# NOTE if you want to revive the CustomMapper, I recommend that you move this functionality over to the CustomMapper class. 

u_train, x_train = rtrnn_trainer.model.convert_dataset(dataloaders['train'].dataset.u, dataloaders['train'].dataset.x)
u_val, x_val = rtrnn_trainer.model.convert_dataset(dataloaders['val'].dataset.u, dataloaders['val'].dataset.x)
train_dataset = c.CustomDataset(u = u_train, x = x_train, device = rtrnn_trainer.model.device)
val_dataset = c.CustomDataset(u = u_val, x = x_val, device = rtrnn_trainer.model.device)
train_loader = DataLoader(train_dataset, batch_size=configs["training_config"]["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=configs["training_config"]["batch_size"], shuffle=False)

In [ ]:
epochs = 200
rtrnn_trainer.train_loop(train_loader, val_loader, epochs = epochs)

In [ ]:
rtrnn_trainer.load_checkpoint(rtrnn_trainer.check_path / "best_model.pth")

For visualization of results &c. go to the expanded notebook. Just basic PCAs and stuff - you can do it. 

# Step 4: HMM models

The full notebook here is [[sim2test.ipynb]]

In [ ]:
x = copy.deepcopy(dataloaders["train"].dataset.x)

dt = 0.5
x[:, :, 1] = x[:, :, 1] * 1.5 * (x[:, :, 1] > 0) + x[:, :, 1] * (-1.5) * (x[:, :, 1] < 0)
hmm_data = PaddedRTSeriesDataset(
    u=dataloaders["train"].dataset.u[0:100], 
    x=x[0:100],
    dt=dt,
    rt_ceiling=1.5,
    keep_onehot=True,
)
hmm_loader = torch.utils.data.DataLoader(
    hmm_data,
    batch_size=16,
    shuffle=True,
)
x_val = copy.deepcopy(dataloaders["val"].dataset.x)
x_val[:, :, 1] = x_val[:, :, 1] * 1.5 * (x_val[:, :, 1] > 0) + x_val[:, :, 1] * (-1.5) * (x_val[:, :, 1] < 0)
hmm_data_val = PaddedRTSeriesDataset(
    u=dataloaders["val"].dataset.u,
    x=x_val,
    dt=dt,
    rt_ceiling=1.5,
    keep_onehot=True,
)
hmm_loader_val = torch.utils.data.DataLoader(
    hmm_data_val,
    batch_size=200,
    shuffle=False,
)

In [ ]:
from Network_models.HMMModels import HMMModel_linearK
K = 8
stim_bits = 3
model = HMMModel_linearK(z_dim = K, stim_types=stim_bits, symmetric=True)
# model.load_checkpoint(MODEL_SAVE_PATH / "HMMModel_linearK" / "best_epoch_no_rt.pth")
model.get_params()
model.to("cuda")
ll_seq = []
epochs = 16
best_ll = -np.inf
model.train()
for epoch in range(epochs): 
    ll_loc = torch.zeros(len(hmm_loader))

    for i, train_set in enumerate(hmm_loader):
        recording = train_set["recording"].to("cuda")
        stim_type = train_set["stimulus_type"].to("cuda")
        mask = train_set["mask"].to("cuda")
        
        ll = model.fit_batch(stim_type, recording, mask.long())
        print(f"train_set {i+1}/{len(hmm_loader)}, log-lik={ll.mean().item():.3f}")
        if ll.mean().item() > best_ll: 
            best_ll = ll.mean().item()
            model.save_checkpoint(MODEL_SAVE_PATH / "HMMModel_linearK" / f"best_epoch.pth")
 
        ll_loc[i] = ll.mean().item()
    ll_seq.append(ll_loc.mean().item())
   
    print(f"Epoch {epoch+1}/{epochs}, log-lik={ll_loc.mean().item():.3f}")
model.eval() 

NOTE: to figure out which stim is assigned to what, go to the figures in the expanded notebook! There is also a cool nx based visualization of the transition matrix. 